In [15]:
# !wget https://www.dropbox.com/s/lnbhc8yuhit4wm5/wikidata5m_alias.tar.gz?dl=1
# !mv wikidata5m_alias.tar.gz?dl=1 wikidata5m_alias.tar.gz
# !tar -zxf wikidata5m_alias.tar.gz
# !rm wikidata5m_alias.tar.gz

In [2]:
!head -n 100 wikidata5m_entity.txt

Q5196650	Cut Your Hair	cut your hair
Q912600	Straumur-Burðarás	Straumur	straumur–burðarás investment bank	straumur	Straumur-Burðarás Investment Bank	straumur-burðarás investment bank	straumur investment bank	straumur-burðarás fjárf.banki	Straumur-Burðarás Fjárf.banki	straumur-burðarás	Straumur Investment Bank	Straumur–Burðarás Investment Bank
Q47551	ditiano	tipciano	titiaen geovene	Tizzianello	Called, Titian Tiziano Vecelli Cavaliere	called titian veccellio	genannt Vecelli Titian	Veccelli Titian genoemd Titiaan	titiano da cadore	ttiziano	dit Le Titien Titianus Vecellio	Tiziano Vecelli	Titiano Vecelli	tissien	Tiziano Vecelli da Cador	titiano vecelli	tizian vecellio	Tiçiano	titiaen	titian (tiziano veccellio)	tetian	ticciano	tiziano veccelli	ziano	Titien Vecelly	Ticciani	titiano vccello	Titien Vecelli da Cador	Tuziano	vicelli	tipziano	tizzani	Tisiaen	Tiziano Vecellio	tiziano veccellio	Tisiano	tishian	Tizano	vecelli	Titian (Tiziano Veccellio)	de Cador Titien Vecelli	Titien de Vecelli	ttali

In [3]:
with open('wikidata5m_entity.txt') as fopen:
    data = fopen.read().split('\n')
    
data[0]

'Q5196650\tCut Your Hair\tcut your hair'

In [4]:
import re
from unidecode import unidecode
from collections import defaultdict
import numpy as np

def cleaning(string):
    string = re.sub('[^A-Za-z ]+', ' ', unidecode(string).lower())
    return re.sub(r'[ ]+', ' ', string).strip()

In [6]:
def get_uniques(string):
    uniques = defaultdict(list)
    splitted = string.split('\t')[1:]
    for s in splitted:
        uniques[cleaning(s)].append(unidecode(s))
        
    r = []  
    for k, v in uniques.items():
        vals = [sum([ord(c) for c in v_]) for v_ in v]
        r.append(v[np.argmin(vals)])
    return r

In [8]:
%%time
get_uniques(data[0])

CPU times: user 84 µs, sys: 56 µs, total: 140 µs
Wall time: 139 µs


['Cut Your Hair']

In [11]:
len(data)

4813492

In [13]:
import ujson as json

filename = 'wikidata5m_entity.txt'
partition_size = 10
limit = len(data) // partition_size
for i in range(partition_size):
    b = data[i * limit: limit + (i * limit)]
    b = [r for r in b if len(r)]
    with open(f'{filename}-{i}', 'w') as fopen:
        json.dump(b, fopen)